## Codes


In [ ]:
!pip install geonamescache
!pip install selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
import pandas as pd

import selenium.webdriver as wd
from selenium.webdriver.common.by import By
import time

import geonamescache

from ipywidgets import widgets

import plotly.express as px

In [ ]:
bar = widgets.IntProgress(
        value=0,
        min=0,
        max=11,
        description='Loading:',
        style={'bar_color': 'gray'}
    )

text = widgets.Text(placeholder='Mumbai', description='City: ')

submit_button = widgets.Button(description='Submit')

In [ ]:
def getWeather(city):
    dfs_t = []
    dfs_h = []
    dfs_w = []
    dfs_p = []
    for i in range(12):
        ccode = gc.search_cities(city)[0]['countrycode']
        cityname = city

        url = 'https://www.wunderground.com/history/monthly/{}/{}/date/2021-{}'.format(ccode, cityname, str(i+1).zfill(2))
        
        co = wd.ChromeOptions()
        co.add_argument('--headless')
        co.add_argument('--no-sandbox')
        co.add_argument('--disable-dev-shm-usage')
        dr = wd.Chrome('chromedriver', options=co)

        dr.get(url)

        time.sleep(10)

        tem = dr.find_element(By.CSS_SELECTOR,
                            'table.days[_ngcontent-app-root-c208] tbody[_ngcontent-app-root-c208] tr[_ngcontent-app-root-c208] td[_ngcontent-app-root-c208]:nth-child(2n)')
        wsp = dr.find_element(By.CSS_SELECTOR,
                            'table.days[_ngcontent-app-root-c208] tbody[_ngcontent-app-root-c208] tr[_ngcontent-app-root-c208] td[_ngcontent-app-root-c208]:nth-child(4n)')
        hum = dr.find_element(By.CSS_SELECTOR,
                            'table.days[_ngcontent-app-root-c208] tbody[_ngcontent-app-root-c208] tr[_ngcontent-app-root-c208] td[_ngcontent-app-root-c208]:nth-child(5n)')
        pre = dr.find_element(By.CSS_SELECTOR,
                            'table.days[_ngcontent-app-root-c208] tbody[_ngcontent-app-root-c208] tr[_ngcontent-app-root-c208] td[_ngcontent-app-root-c208]:nth-child(7n)')

        df_t = pd.DataFrame([eval(q.split()[1]) for q in tem.text.split('\n')[1:]])
        df_w = pd.DataFrame([eval(q.split()[1]) for q in wsp.text.split('\n')[1:]])
        df_h = pd.DataFrame([eval(q.split()[1]) for q in hum.text.split('\n')[1:]])
        df_p = pd.DataFrame([eval(q.split()[0]) for q in pre.text.split('\n')[1:]])

        dfs_t.append(df_t)
        dfs_w.append(df_w)
        dfs_h.append(df_h)
        dfs_p.append(df_p)

        dr.quit()
        
        bar.value = i
        
    temp = pd.concat(dfs_t)
    wind = pd.concat(dfs_w)
    humi = pd.concat(dfs_h)
    prec = pd.concat(dfs_p)
    
    temp.columns = ['Temperature (°F)']
    wind.columns = ['Wind (mph)']
    humi.columns = ['Humidity (%)']
    prec.columns = ['Precipitation (in)']

    for x in [temp, wind, humi, prec]:
        x.reset_index(drop=True, inplace=True)
        x.index = pd.date_range(start='2021-01', periods=len(x), freq='D', name='Date')
    
    return [temp, wind, humi, prec]

In [ ]:
cityx = []
gc = geonamescache.GeonamesCache()

def on_button_click(b):
    cityx.append(text.value)
    display(bar)
    city = cityx[-1]
    temp, wind, humi, prec = getWeather(city.capitalize())
    px.line(data_frame=temp, y='Temperature (°F)', color_discrete_sequence=['grey'],
            title='Average Temperature of {}, {} in 2021'.format(city.capitalize(), gc.search_cities(city.capitalize())[0]['countrycode'].upper())).show()
    px.line(data_frame=wind, y='Wind (mph)', color_discrete_sequence=['grey'],
            title='Average Wind Speed of {}, {} in 2021'.format(city.capitalize(), gc.search_cities(city.capitalize())[0]['countrycode'].upper())).show()
    px.line(data_frame=humi, y='Humidity (%)', color_discrete_sequence=['grey'],
            title='Average Humidity of {}, {} in 2021'.format(city.capitalize(), gc.search_cities(city.capitalize())[0]['countrycode'].upper())).show()
    px.line(data_frame=prec, y='Precipitation (in)', color_discrete_sequence=['grey'],
            title='Average Precipitation of {}, {} in 2021'.format(city.capitalize(), gc.search_cities(city.capitalize())[0]['countrycode'].upper())).show()

In [ ]:
submit_button.on_click(on_button_click)

In [ ]:
toShow = widgets.HBox([text, submit_button])

# Execution


In [ ]:
toShow